In [1]:
from utils.df_handle import *

Default bq project:  spatial-vision-343005.biteam


In [2]:
# # from utils.df_handle import *
# import pendulum
# from airflow import DAG
# from airflow.operators.dummy_operator import DummyOperator
# from airflow.operators.python_operator import PythonOperator

# local_tz = pendulum.timezone("Asia/Bangkok")

# name='FILES'
# prefix='UPLOAD_'
# csv_path = '/usr/local/airflow/plugins'+'/'

# dag_params = {
#     'owner': 'airflow',
#     "depends_on_past": False,
#     'start_date': datetime(2022, 5, 10, tzinfo=local_tz),
#     'email_on_failure': True,
#     'email_on_retry': False,
#     'email':['duyvq@merapgroup.com', 'vanquangduy10@gmail.com'],
#     'do_xcom_push': False,
#     'execution_timeout':timedelta(seconds=300)
#     # 'retries': 3,
#     # 'retry_delay': timedelta(minutes=10),
# }

# dag = DAG(prefix+name,
#           catchup=False,
#           default_args=dag_params,
#           schedule_interval= '*/30 6-23 * * *',
#           tags=[prefix+name, 'Sync', '30mins']
# )

In [3]:
# import xlsxwriter

In [4]:
# csv_path = '/usr/local/airflow/plugins'+'/'
# csv_path = ''
# prefix = ''
# name = ''

In [5]:
sql_1 = \
"""
select
GETDATE() as ThoiGianRutBaoCao,
pd.Crtd_DateTime as NgayTaoYCRH,
pd.BranchID as ChiNhanh,
pd.PONbr as YCRH,
LineRef as DongTrenPo,
LoaiNhapHang = 'HangTonKho',
pd.InvtID as MaSanPham,
pd.TranDesc as DienGiai,
pd.SiteID as MaKho,
ist.Name as TenKho,
ph.VendID as MaNhaCC,
ph.Status,
QtyOrd SLGoc,
QtyRcvd SLThucNhap,
QtyOrd-QtyRcvd as SLDatHang
from PO_Detail pd
INNER JOIN PO_Header ph on
pd.PONbr = ph.PONbr and
pd.BranchID = ph.BranchID
and ph.status in ('M','O')
INNER JOIN IN_Site ist ON
pd.SiteID = ist.SiteId
"""

In [6]:
df1 = get_ms_df(sql_1)

In [7]:
# df1.to_csv(csv_path+f'{prefix}{name}test.csv', index=False)

In [8]:
# df1.head()

In [9]:
assert checkdup(df1, 2, ['ChiNhanh', 'YCRH','MaSanPham']).sum() == 0, "Duplicate found"

In [10]:
sql_2 = \
"""
select
GETDATE() as ThoiGianRutBaoCao,
pt.Crtd_DateTime as NgayTaoHoaDon,
pr.BranchID as ChiNhanh,
case when pt.PONbr = '' then 'KHONGCOYCRH' else pt.PONbr end as YCRH,
pt.BatNbr as SoHoaDon,
pr.Status,
pt.InvtID as MaSanPham,
pt.RcptQty as SoLuongChoNhap
from PO_Trans pt
INNER JOIN dbo.PO_Receipt pr ON pr.BatNbr = pt.BatNbr AND pr.BranchID = pt.BranchID and pr.Status = 'H'
"""

In [11]:
df2 = get_ms_df(sql_2)

In [12]:
# df2.head()

In [13]:
df2.YCRH.fillna("KHONGCOYCRH", inplace=True)

In [14]:
df2.to_excel("hoadonnhaptam.xlsx", sheet_name="Duy", index=False)

In [15]:
df2 = pivot(df2, ['ChiNhanh','YCRH','MaSanPham'], {'SoLuongChoNhap':np.sum})

In [16]:
# df2.head()

In [17]:
df3 = df1.merge(df2, how='left', on=['ChiNhanh','YCRH','MaSanPham'])

In [18]:
df3.SoLuongChoNhap.fillna(0, inplace=True)

In [19]:
df3['SoLuongChoNhapConLai'] = df3.SLDatHang - df3.SoLuongChoNhap

In [20]:
df3['CanXemXet'] = np.where(df3['SoLuongChoNhapConLai'] > 0, "YES", "NO")

In [21]:
dk = df3['CanXemXet'] == "YES"

In [22]:
df4 = df3[dk]

In [23]:
df4.to_excel("output.xlsx", sheet_name="Duy", index=False)

In [27]:
from google.oauth2 import service_account
from googleapiclient import discovery
from googleapiclient.http import MediaFileUpload

In [32]:
scopes = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive',
          'https://www.googleapis.com/auth/drive.file']
jsonfile = 'D:/data_sale/datateam1599968716114-6f9f144b4262.json'
credentials = service_account.Credentials.from_service_account_file(jsonfile, scopes=scopes)
service = discovery.build('drive', 'v3', credentials=credentials)

In [33]:
folder_id = '1s6-KRfQpFNTZINO0au9ETcEbqxfi_gp7'
file_name = 'output.xlsx'
file_type = 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
file_body={'name':file_name,'addParents':[folder_id]}
query = f"parents = '{folder_id}'"
files = service.files().list(q=query).execute()
fileid1  = files['files'][0]['id']
media = MediaFileUpload('{0}'.format(file_name), mimetype=file_type)
updated_file = service.files().update(fileId=fileid1, body=file_body, media_body=media)
updated_file.execute()

{'kind': 'drive#file',
 'id': '1z_TbNPWG6Y01iwoQioAzMpBXruBHTSZC',
 'name': 'output.xlsx',
 'mimeType': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'}

In [34]:
folder_id = '1s6-KRfQpFNTZINO0au9ETcEbqxfi_gp7'
file_name = 'hoadonnhaptam.xlsx'
file_type = 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
file_body={'name':file_name,'addParents':[folder_id]}
query = f"parents = '{folder_id}'"
files = service.files().list(q=query).execute()
fileid2  = files['files'][1]['id']
media = MediaFileUpload('{0}'.format(file_name), mimetype=file_type)
updated_file = service.files().update(fileId=fileid2, body=file_body, media_body=media)
updated_file.execute()

{'kind': 'drive#file',
 'id': '1OYwXwEt_wZbBK9uTjL1717z4YY838ZtL',
 'name': 'hoadonnhaptam.xlsx',
 'mimeType': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'}

In [31]:
dummy_start = DummyOperator(task_id="dummy_start", dag=dag)

insert = PythonOperator(task_id="insert", python_callable=insert, dag=dag)

update = PythonOperator(task_id="update", python_callable=update, dag=dag)

update_sync_dms_ardoc_2 = PythonOperator(task_id="update_sync_dms_ardoc_2", python_callable=update_sync_dms_ardoc_2, dag=dag)

dummy_start >> update >> insert >> update_sync_dms_ardoc_2

NameError: name 'DummyOperator' is not defined